In [1]:
# import libraries
import os
import numpy as np
from tqdm import tqdm

from ifum_stitch import Stitch
from ifum_maskopt import Mask
from ifum_rectify import Rectify
from ifum_calibrate import Calibrate
from ifum_stack import Stack

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# directory containing unprocessed files

# directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240212\\"
directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240210\\"

# all files included in a single stack, repeat where necessary
# only include string in file that includes all files from single exposure

# data_filenames = ["1156","1157","1158","1162","1163","1164","1168","1169","1170"]
# arc_filenames = ["1161","1161","1161","1167","1167","1167","1172","1172","1172"]
# flat_filenames = ["1160","1160","1160","1166","1166","1166","1171","1171","1171"]
data_filenames = ["0721","0722","0723","0727","0728","0729","0736","0737","0738"]
arc_filenames = ["0725","0725","0725","0733","0733","0733","0740","0740","0740"]
flat_filenames = ["0724","0724","0724","0734","0734","0734","0739","0739","0739"]

# mask???
# mode LR,STD,HR
mode = "STD"

# bad masks (on scale 1-276)
bad_blues = [23]
bad_reds = []

# stars to use in WCS (list RA,Dec)
# all stars should be present in at least some dithers

# wcs_stars = [[74.8322, -58.6579],
#              [74.8279, -58.6548],
#              [74.8314, -58.6526],
#              [74.8254, -58.6572],
#              [74.8303, -58.6543]]
wcs_stars = [[74.8322, -58.6579],
             [74.8305, -58.6603],
             [74.8308, -58.6587],
             [74.8254, -58.6572],
             [74.8237, -58.6594]]


# things to add...
# plot = False (generic plots)
# debug_mode = False (extra print statements, plots, etc.)
# overwrite = False (if file exists, assume already processed correctly)

In [3]:
# assert statements!!!
bad_masks = [np.array(bad_blues)-1,np.array(bad_reds)-1]
wcs_stars = np.array(wcs_stars)
if mode == "STD":
    total_masks = 552
    mask_groups = 12
else:
    print("invalid mode")

<h1><strong><span style="color:purple">STITCH</h1>

<span style="color:orange"><strong>creates 2 files, one for each detector, for both the data and arc files. also produces a cosmic ray mask for the data.

In [4]:
# out directory where all files are stored
if not os.path.exists("out"):
    os.makedirs("out")

# stitch and create file
for file in tqdm(data_filenames+arc_filenames+flat_filenames):
    file_to_stitch = Stitch(directory,file,None,None,None,None,None)
    file_to_stitch.load_files()
    file_to_stitch.save_file()
print("stitched files saved")

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [00:24<00:00,  1.11it/s]

stitched files saved


In [5]:
# use flat and median filter to get rid of internal bias
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_bias = Stitch(directory,None,None,"b",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
    file_for_bias = Stitch(directory,None,None,"r",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
print("internal bias solved")

100%|██████████| 9/9 [00:32<00:00,  3.65s/it]

internal bias solved


In [4]:
# create cosmic ray masks
for datafilename in tqdm(data_filenames):
    file_for_cmray = Stitch(directory,None,None,"b",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
    file_for_cmray = Stitch(directory,None,None,"r",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
print("cosmic ray masks created")

100%|██████████| 9/9 [15:07<00:00, 100.86s/it]

cosmic ray masks created


<h1><strong><span style="color:purple">OPTIMIZE MASK</h1>

<span style="color:orange"><strong>use current mask for single gaussian fits to get better mask guess  

In [5]:
# first guess; complex guess
# IMPLEMENT RANSAC??? (HBDSCAN?)
for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

100%|██████████| 3/3 [37:42<00:00, 754.29s/it]


In [6]:
# change the degrees until fully confident with all the fits!
center_deg = 5
sigma_deg = 3

for flatfilename in np.unique(flat_filenames):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)    

In [ ]:
# TODO: calculate sig_mult based on mask non-overlap
sig_mult = 1.5

for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)

100%|██████████| 3/3 [05:49<00:00, 116.63s/it]


In [8]:
# optimize arc files
expected_peaks = 15
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks)

100%|██████████| 3/3 [41:16<00:00, 825.39s/it]


In [9]:
# optimize data files
expected_peaks = 25
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks)
    file_for_mask.get_rots(arcfilename,datafilename)

100%|██████████| 9/9 [3:05:37<00:00, 1237.49s/it]  


In [10]:
# create masks for data
sig_mult = 1.5

for datafilename, flatfilename in tqdm(zip(data_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename)

100%|██████████| 9/9 [25:19<00:00, 168.81s/it]


In [11]:
# create masks for arc!
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)

100%|██████████| 3/3 [07:21<00:00, 147.24s/it]


<h1><strong><span style="color:purple">RECTIFY</h1>

<span style="color:orange"><strong>rectify + calibrate

In [12]:
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data",sig_mult)
    
    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("data",sig_mult)

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",sig_mult)    

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",sig_mult)    

100%|██████████| 3/3 [05:55<00:00, 118.54s/it]


In [13]:
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

    file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

  0%|          | 0/9 [00:00<?, ?it/s]

BAD emission line


 11%|█         | 1/9 [00:08<01:10,  8.86s/it]

BAD emission line


 22%|██▏       | 2/9 [00:17<01:00,  8.60s/it]

...

 78%|███████▊  | 7/9 [00:57<00:16,  8.02s/it]

BAD emission line


 89%|████████▉ | 8/9 [01:06<00:08,  8.38s/it]

BAD emission line


100%|██████████| 3/3 [00:13<00:00,  4.48s/it]


In [14]:
# calibrate (rectified xs to wls)
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.calib()

100%|██████████| 9/9 [04:00<00:00, 26.75s/it]


In [15]:
# viz
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

100%|██████████| 9/9 [03:55<00:00, 26.16s/it]


<h1><strong><span style="color:purple">CALIBRATE</h1>

<span style="color:orange"><strong>calibrate

In [16]:
sig_mult = 1.5
bins = np.arange(7500,10000,1)

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.get_spectra(sig_mult,bins,color="b")
    file_for_calib.get_spectra(sig_mult,bins,color="r")

100%|██████████| 9/9 [4:51:35<00:00, 1943.90s/it]  


In [17]:
# intenisty calibrate all spectra, combine blue and red amplifiers

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib.intensity_corr()

100%|██████████| 9/9 [05:10<00:00, 34.55s/it]


In [ ]:
# ONLY VIZ

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

<h1><strong><span style="color:purple">STACK</h1>

<span style="color:orange"><strong>stack dithers

In [18]:
# individually create datacubes to be used in dither stack

files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.hex_to_grid()
files_for_stack.spectra_to_datacube()

hexagon grid: 99.000,88.777
pixel grid: 99,89
8811 pixels


overlap percentages: 100%|██████████| 8811/8811 [00:33<00:00, 263.25it/s]


8691 (98.638%) of pixels overlap with hexagon grid
8183 (92.873%) of pixels overlap fully with hexagon grid


graphing pixels: 100%|██████████| 8811/8811 [00:12<00:00, 699.24it/s]


plotting...


graphing pixels: 100%|██████████| 8811/8811 [00:13<00:00, 632.08it/s]


plotting...


100%|██████████| 99/99 [02:24<00:00,  1.46s/it]


In [4]:
# transform data to WCS using cross-correlation
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.wcs_datacubes()

calculting 2D cross-correlation shift guesses...
select 5 WCS stars in order; see popup window
optimizing WCS star coordinates...
performing WCS transforms...
	0721 used 5 reference stars
	0722 used 5 reference stars
	0723 used 5 reference stars
	0727 used 5 reference stars
	0728 used 5 reference stars
	0729 used 5 reference stars
	0736 used 5 reference stars
	0737 used 5 reference stars
	0738 used 5 reference stars
WCS transformations complete


In [5]:
# calibrate intensity for all datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.full_intensity_callibration()

In [6]:
# stack datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars)
files_for_stack.stack_datacubes()

100%|██████████| 99/99 [1:07:33<00:00, 40.94s/it]


final data cubes saved!
